In [43]:
import random
import json
import urllib.request
import collections
import time

#Константы:
USERS = "elmiram, maryszmary, lizaku, nevmenandr, ancatmara, roctbb, JelteF, gvanrossum"
USERS_D = USERS.split(', ')


# С этой функции начинается программа, на выходе дает имя пользователя из списка
def userChoice():
    # Берем базу, предложенную в задании и делаем из нее список
    user_choice = input('Введите имя пользователя из списка {}: '.format(USERS))
    # Пользователь вводит имя пользователя гитхаба
    while True:
        if user_choice in USERS_D:
            print('Вы выбрали пользователя {}'.format(user_choice))
            break
        else:
            # Для разнообразных реакций программы, возьмем их из отдельного файла
            with open('lexicon.txt', 'r', encoding="utf-8") as f:
                f = f.readlines()
            print(random.choice(f))
            user_choice = input('Введите имя пользователя: ')
            
    return user_choice

In [44]:
# Эта функция берет на вход имя отдельного пользователя и возвращаяет json-объект с данными об аккаунте
def getData(user):
    time.sleep(2)
    #ПУСТЫЕ ТОКЕНЫ ВСТАВЬТЕ СВОИ
    token = ""
    # Листаем страницы, добавляем в переменную со строкой новые страницы и проверяем, не пустая ли она
    page = 1
    url = 'https://api.github.com/users/%s/repos?page=%s&per_page=100&access_token=%s' % (user, page, token)
    response = urllib.request.urlopen(url)
    text = response.read().decode('utf-8')
    
    data = json.loads(text)
    
    while len(data) == 100:
        page += 1
        url = 'https://api.github.com/users/%s/repos?page=%s&per_page=100&access_token=%s' % (user, page, token)
        response = urllib.request.urlopen(url)
        text = response.read().decode('utf-8')
        data_new = json.loads(text)
        data = data + data_new
    
    return data    
    

In [45]:
#На вход - данные json-объекта, возвращает названия репозиториев пользователя и их описания

def userDescription(data):
    print('\nСписок репозиториев пользователя: ')
    n = 0
    for i in data:
        n += 1
        if n == len(data):
            print('{} : {}'.format(i["name"], i["description"]), end='\n')
        else:
            print('{} : {}'.format(i["name"], i["description"]), end=',\n')
    return data

In [46]:
# Функция берет данные о пользователе и ищет языки

def userRepLang(data):
    languages = {}
    for i in data:
        time.sleep(2)
    # На всякий случай не будем считать за язык None
        if i["language"] != None:
            if i["language"] in languages:
                languages[i["language"]].append(i["name"])
            else:
                languages[i["language"]] = []
                languages[i["language"]].append(i["name"])
        
    n = 0                                    # Создадим счетчик, чтобы написать "и" перед последним языком

    #Имя пользователя в другой функции, поэтому найдем его на странице   
    print('\nПользователь {} пишет на '.format(i["full_name"].split('/')[0]), end='')    
    for key, value in languages.items():
        n += 1
        if n == len(languages) - 1:
            print(key, end=' и ')
        elif n == len(languages):
            print(key, end='. \n\n')
        else:
            print(key, end=', ')
            
    for key, value in languages.items():
        if len(value) == 1:
            print('Язык {} встречается в {} репозитории.'.format(key, len(value)))
        else:
            print('Язык {} встречается в {} репозиториях.'.format(key, len(value)))

In [47]:
# Функци берет данные о пользователе и создает частотный словарь для языков программирования

def userLang():
    lang = []
    for i in USERS_D:
        data = getData(i)
        for i in data:
            lang.append(i["language"])
    print('Самый популярный язык среди пользователей из списка {}:'.format(USERS))
    lang = collections.Counter(lang)
    for key, value in lang.items():
        if value == max(lang.values()):
            print('{}\t--\t{}'.format(key, value), end='\n')

            

In [48]:
# Функция берет данные о пользователе и создает словарь для количества репозиториев

def compareRep():
    rep = {}
    winner = []
    n = 0
    
    # Для каждого пользователя получаем данные из функции getData()
    for i in USERS_D:
        time.sleep(2)
        data = getData(i)
        rep[i] = len(data)
        
        # Находим самое большое количество репозиториев
    for key, reps in rep.items():
        if reps == max(rep.values()):
            winner.append(key) 
    if len(winner) == 1:
        print("\nИз списка {} больше всего репозиториев у пользователя".format(USERS), end=' ')
    else:
        print("\nИз списка {} больше всего репозиториев у пользователей".format(USERS), end=' ')
        # Алгоритм, чтобы ставить перед последним однородным членом союз "и" и запятые
    for key in winner:
        n += 1
        if n == len(winner) - 1:
            print(key,  end=' и ')
        elif n == len(winner):
            print(key, end='. \n\n')
        else:
            print(key, end=', ')
                
    

In [49]:
# Функция берет данные о пользователях по новой ссылке и создает словарь с количеством подписчиков

def popular():
    followers = {}
    winner = []
    for user in USERS_D:
        time.sleep(2)
        #ПУСТЫЕ ТОКЕНЫ ВСТАВЬТЕ СВОИ
        token = ""
        # Листаем страницы
        page = 1
        url = 'https://api.github.com/users/%s/followers?page=%s&per_page=100&access_token=%s' % (user, page, token)
        response = urllib.request.urlopen(url)
        text = response.read().decode('utf-8')
            
        data = json.loads(text)
        followers[user] = len(data)
        while len(data) == 100:
            url = 'https://api.github.com/users/%s/followers?page=%s&per_page=100&access_token=%s' % (user, page, token)
            page += 1
            response = urllib.request.urlopen(url)
            text = response.read().decode('utf-8')
            
            data = json.loads(text)
            followers[user] += len(data)
    for key, value in followers.items():
        if value == max(followers.values()):
            winner.append(key) 
            print(key, value)
    if len(winner) == 1:
        print("\nИз списка {} больше всего подписчиков у пользователя".format(USERS), end=' ')
    else:
        print("\nИз списка {} больше всего подписчиков у пользователей".format(USERS), end=' ')
        
        # Алгоритм, чтобы ставить перед последним однородным членом союз "и" и запятые
    n = 0
    for key in winner:
        n += 1
        if n == len(winner) - 1:
            print(key, end=' и ')
        elif n == len(winner):
            print(key, end='. \n\n')
        else:
            print(key, end=', ')
            

In [ ]:
def main():
    return userRepLang(userDescription(getData(userChoice()))), compareRep(), userLang(), popular()

if __name__ == "__main__":
    main()


Введите имя пользователя из списка elmiram, maryszmary, lizaku, nevmenandr, ancatmara, roctbb, JelteF, gvanrossum: JelteF
Вы выбрали пользователя JelteF

Список репозиториев пользователя: 
3akai-embedanything-widget : None,
3akai-exhibit-widget : None,
3akai-handsontable-widget : Handsontable widget for Sakai OAE,
3akai-timeglider-widget : A timeline widget for Sakai OAE using Timeglider,
3akai-twitter-widget : A twitter feed widget for Sakai OAE,
3akai-ux : Front-end development for Sakai 3,
ace : Ace (Ajax.org Cloud9 Editor),
ale : Asynchronous Lint Engine,
algorithms : Algorithms Schoolwork,
AppStudio_HelloWorld : Hello world application,
archiver : Easily create and extract .zip, .tar, .tar.gz, .tar.bz2, .tar.xz, .tar.lz4, .tar.sz, and .rar (extract-only) files with Go,
awesome-go : A curated list of awesome Go frameworks, libraries and software,
baas32 : An alternative Python implementation of Douglas Crockford's base32 encoding scheme,
babel-plugin-flow-react-proptypes :  A babel


Пользователь JelteF пишет на JavaScript, Vim script, TeX, Java, Go, Python, CSS, C, Rust, Shell, C++, OCaml, Ruby, Matlab, CoffeeScript, VimL, HTML, Makefile, AGS Script, KiCad и PHP. 

Язык JavaScript встречается в 20 репозиториях.
Язык Vim script встречается в 3 репозиториях.
Язык TeX встречается в 3 репозиториях.
Язык Java встречается в 6 репозиториях.
Язык Go встречается в 19 репозиториях.
Язык Python встречается в 37 репозиториях.
Язык CSS встречается в 4 репозиториях.
Язык C встречается в 5 репозиториях.
Язык Rust встречается в 10 репозиториях.
Язык Shell встречается в 13 репозиториях.
Язык C++ встречается в 4 репозиториях.
Язык OCaml встречается в 1 репозитории.
Язык Ruby встречается в 1 репозитории.
Язык Matlab встречается в 1 репозитории.
Язык CoffeeScript встречается в 1 репозитории.
Язык VimL встречается в 4 репозиториях.
Язык HTML встречается в 1 репозитории.
Язык Makefile встречается в 1 репозитории.
Язык AGS Script встречается в 1 репозитории.
Язык KiCad встречается в 1 